tgb - 10/15/2019 - Drafting script to calculate diurnally-averaged shortwave heating for the ITCZ

In [1]:
import util.Landau_Potential_Diffusion as Landau
import util.curvature as curve
import util.pdf as PDF

import xarray as xr
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime
from skimage import measure
from scipy import ndimage
import scipy.integrate as sin
from scipy.optimize import curve_fit

import pickle
import sys

In [18]:
path = '/nfs/twcroninlab002/tbeucler/ERA5/ERA5_LHF_SHF/'
path_PKL = '/nfs/twcroninlab002/tbeucler/2019_WMI/dev/tbeucler/PKL_DATA/'

In [3]:
RAD = xr.open_mfdataset(path+'????/??RAD.nc',combine='by_coords')

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [5]:
dt = 3600
tcoor = dt*np.arange(0,RAD.time.shape[0])

In [8]:
latmin = -30
latmax = 30
lonmin = 300
lonmax = 360

lonAtl = RAD.longitude.sel({'longitude':slice(lonmin,lonmax)})
latAtl = RAD.latitude.sel({'latitude':slice(latmax,latmin)})

In [9]:
SWAtl = RAD['tsr'].sel({'longitude':slice(lonmin,lonmax),'latitude':slice(latmax,latmin)})/3600-\
RAD['ssr'].sel({'longitude':slice(lonmin,lonmax),'latitude':slice(latmax,latmin)})/3600

In [10]:
YEAR = np.linspace(2000,2018,19).astype(int)
print(YEAR)

[2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018]


In [16]:
SWD = {}
print('year=',year)
date1 = str(year)+'-01-01T00:00:00'
date2 = str(year)+'-12-31T23:00:00'
it_tot = PDF.index_date(SWAtl.time,date1)[0]
Nt = SWAtl.time.sel({'time':slice(date1,date2)}).shape[0]
for i,year in enumerate(YEAR[:-1]):
    print('i=',i,' & year=',year)
    date1 = str(year)+'-01-01T00:00:00'
    date2 = str(year)+'-12-31T23:00:00'
    Nt = SWAtl.time.sel({'time':slice(date1,date2)}).shape[0]
    SWD[str(year)] = np.zeros((Nt,))
    for it in range(Nt):
        print('it=',it,' & it_tot=',it_tot,'               ',end='\r')
        if it_tot>12: SWD[str(year)][it] = SWAtl[it_tot-12:(it_tot+12),:,:].mean(dim=('longitude','latitude','time')).values
        else: SWD[str(year)][it] = SWAtl[it_tot:(it_tot+24),:,:].mean(dim=('longitude','latitude','time')).values
        it_tot+=1

year= 2000
i= 0  & year= 2000


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/tbeucler/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-b2d71b77c65e>", line 16, in <module>
    else: SWD[str(year)][it] = SWAtl[it_tot:(it_tot+24),:,:].mean(dim=('longitude','latitude','time')).values
  File "/home/tbeucler/miniconda3/lib/python3.7/site-packages/xarray/core/dataarray.py", line 488, in values
    return self.variable.values
  File "/home/tbeucler/miniconda3/lib/python3.7/site-packages/xarray/core/variable.py", line 389, in values
    return _as_array_or_item(self._data)
  File "/home/tbeucler/miniconda3/lib/python3.7/site-packages/xarray/core/variable.py", line 210, in _as_array_or_item
    data = np.asarray(data)
  File "/home/tbeucler/miniconda3/lib/python3.7/site-packages/numpy/core/numeric.py", line 538, in asarray
    return array(a, dtype, copy=False, order=order)
  File "/home/tbeucler

KeyboardInterrupt: 

In [19]:
# Save the contour length in a pickle file
hf = open(path_PKL+'10_15_SWD'+str(year)+'.pkl','wb')
SWDdata = {"SWD":SWD,"time":SWAtl.time}
pickle.dump(SWDdata,hf)
hf.close()

In [1]:
year = 2009

import util.Landau_Potential_Diffusion as Landau
import util.curvature as curve
import util.pdf as PDF

import xarray as xr
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime
from skimage import measure
from scipy import ndimage
import scipy.integrate as sin
from scipy.optimize import curve_fit

import pickle
import sys

path = '/nfs/twcroninlab002/tbeucler/ERA5/ERA5_LHF_SHF/'
path_PKL = '/nfs/twcroninlab002/tbeucler/2019_WMI/dev/tbeucler/PKL_DATA/'

RAD = xr.open_mfdataset(path+'????/??RAD.nc',combine='by_coords')

dt = 3600
tcoor = dt*np.arange(0,RAD.time.shape[0])

latmin = -30
latmax = 30
lonmin = 300
lonmax = 360

lonAtl = RAD.longitude.sel({'longitude':slice(lonmin,lonmax)})
latAtl = RAD.latitude.sel({'latitude':slice(latmax,latmin)})

SWAtl = RAD['tsr'].sel({'longitude':slice(lonmin,lonmax),'latitude':slice(latmax,latmin)})/3600-\
RAD['ssr'].sel({'longitude':slice(lonmin,lonmax),'latitude':slice(latmax,latmin)})/3600

YEAR = np.linspace(2000,2018,19).astype(int)
print(YEAR)

SWD = {}
print('year=',year)
date1 = str(year)+'-01-01T00:00:00'
date2 = str(year)+'-12-31T23:00:00'
it_tot = PDF.index_date(SWAtl.time,date1)[0]
Nt = SWAtl.time.sel({'time':slice(date1,date2)}).shape[0]

print('year=',year)
date1 = str(year)+'-01-01T00:00:00'
date2 = str(year)+'-12-31T23:00:00'
Nt = SWAtl.time.sel({'time':slice(date1,date2)}).shape[0]
SWD[str(year)] = np.zeros((Nt,))
for it in range(Nt):
    print('it=',it,' & it_tot=',it_tot,'               ',end='\r')
    if it_tot>12: SWD[str(year)][it] = SWAtl[it_tot-12:(it_tot+12),:,:].mean(dim=('longitude','latitude','time')).values
    else: SWD[str(year)][it] = SWAtl[it_tot:(it_tot+24),:,:].mean(dim=('longitude','latitude','time')).values
    it_tot+=1
        
# Save the contour length in a pickle file
hf = open(path_PKL+'10_15_SWD'+str(year)+'.pkl','wb')
SWDdata = {"SWD":SWD,"time":SWAtl.time}
pickle.dump(SWDdata,hf)
hf.close()

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018]
year= 2009
year= 2009


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [2]:
# Save the contour length in a pickle file
hf = open(path_PKL+'10_15_SWD'+str(year)+'.pkl','wb')
SWDdata = {"SWD":SWD,"time":SWAtl.time}
pickle.dump(SWDdata,hf)
hf.close()